In [35]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [36]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [37]:
from tensorflow.keras.datasets import mnist

In [38]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [39]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [40]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [41]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [42]:
model = Sequential()

In [43]:
model.add(Conv2D(16, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [44]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [45]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [46]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [47]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [48]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [49]:
model.add(Flatten())

In [50]:
model.add(Dense(54, activation='relu'))

In [51]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

#### 組裝

In [53]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [54]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0898 - accuracy: 0.1853
Epoch 2/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0893 - accuracy: 0.3015
Epoch 3/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0885 - accuracy: 0.3702
Epoch 4/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0853 - accuracy: 0.3940
Epoch 5/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0571 - accuracy: 0.6287
Epoch 6/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0235 - accuracy: 0.8521
Epoch 7/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0149 - accuracy: 0.9044
Epoch 8/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0115 - accuracy: 0.9271
Epoch 9/12
469/469 [==============================] - 2s 3ms/step - loss: 0.0095 - accuracy: 0.9395
Epoch 10/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0083 - accuracy: 0.9473

### Step 3. 預測

In [55]:
result = model.predict_classes(x_test)

### 看看測試資料表現如何

In [56]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0060 - accuracy: 0.9621


In [57]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 96.21%


In [58]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [59]:
from ipywidgets import interact_manual

In [60]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [61]:
model.save('myCNNmodel.h5')